In [2]:
import pandas as pd
from sqlalchemy import create_engine
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import SQLDatabase

from llama_index.core.prompts import BasePromptTemplate, PromptTemplate


In [3]:
file_path = "data/out/recipes_flat.json"
df = pd.read_json(file_path)
llm_gpt = OpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
)
# embed_gpt = OpenAIEmbedding(model='text-embedding-3-large')


In [4]:
engine = create_engine("sqlite:///data/out/recipes.db")

sql_database = SQLDatabase(
    engine=engine, include_tables=["recipes"], sample_rows_in_table_info=2
)


We need to tune the model because the LLM can return aswers that are not related to the question. We can use a simple heuristic to filter out these answers. We can check if the answer contains any of the words in the question. If it does, we can discard the answer. This heuristic is not perfect, but it can help to improve the performance of the model. we can modify the model for the query search and the response too


In [53]:
PROMPT_MODEL = """
'Given an input question, synthesize a response from the query results.

If the question is about a recipe, provide the recipe details in a friendly and helpful manner.
If the answer is not in the database, respond kindly with "I’m sorry, I don't have that information at the moment, but I’d love to help you with something else."
If the question is not about a recipe, respond warmly with "I specialize in recipes, and I'd be delighted to assist you with any culinary questions you have!"

You are a friendly and gentle chatbot with a warm personality.

Query: {query_str}
SQL: {sql_query}
SQL Response: {context_str}
Response: '
"""


response_synthesis_prompt = PromptTemplate(
    template=PROMPT_MODEL,
)


In [115]:
## SQL prompt template
SQL_PROMPT_MODEL = """
Given an input question, first identify relevant keywords and possible synonyms based on the context of the user's question and the database schema, column names, or categories. Use many key words for the search if necessary. Then create a syntactically correct {dialect} query to run. After looking at the results of the query, return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
Keywords and Synonyms: Identified keywords and synonyms
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

You are a friendly and gentle chatbot with a warm personality. Always respond in a kind and helpful manner.

Question: {query_str}
Keywords and Synonyms:
SQLQuery:
"""

text_to_sql_prompt = PromptTemplate(template=SQL_PROMPT_MODEL, type="text-to-sql")


In [116]:
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["recipes"],
    llm=llm_gpt,
    response_synthesis_prompt=response_synthesis_prompt,
    text_to_sql_prompt=text_to_sql_prompt,
)


In [119]:
query_engine.get_prompts()["sql_retriever:text_to_sql_prompt"]


PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['dialect', 'schema', 'query_str'], kwargs={'type': 'text-to-sql'}, output_parser=None, template_var_mappings=None, function_mappings=None, template="\nGiven an input question, first identify relevant keywords and possible synonyms based on the context of the user's question and the database schema, column names, or categories. Use many key words for the search if necessary. Then create a syntactically correct {dialect} query to run. After looking at the results of the query, return the answer. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Al

In [120]:
query_str = "que receta me recomiendas si deseo bajar de peso?"
response = query_engine.query(query_str)
response.response


'¡Hola! Si estás buscando recetas saludables para bajar de peso, aquí te dejo algunas recomendaciones deliciosas:\n\n1. **Receta De Ceviche De Chayote**\n   - **Tiempo de preparación:** 15 mins\n   - **Tiempo de cocción:** 5 mins\n   - **Dificultad:** Baja\n\n2. **Coliflor Rostizada Con Dip Cremoso**\n   - **Tiempo de preparación:** 20 mins\n   - **Tiempo de cocción:** 35 mins\n   - **Dificultad:** Baja\n\n3. **Ensalada Con Granada Pollo Y Nuez**\n   - **Tiempo de preparación:** 20 mins\n   - **Tiempo de cocción:** 15 mins\n   - **Dificultad:** Baja\n\n4. **Tacos De Lechuga Con Atun**\n   - **Tiempo de preparación:** 10 mins\n   - **Tiempo de cocción:** N/A\n   - **Dificultad:** Baja\n\n5. **Omelette De Claras Con Verduras**\n   - **Tiempo de preparación:** 20 mins\n   - **Tiempo de cocción:** 15 mins\n   - **Dificultad:** Baja\n\nEstas recetas son no solo saludables, sino también fáciles de preparar. ¡Espero que disfrutes cocinando y comiendo estas opciones! Si necesitas más ayuda o d

In [113]:
print(response.metadata["sql_query"])


SELECT name, prep_time, cook_time, difficulty FROM recipes WHERE category LIKE '%saludable%' ORDER BY difficulty ASC LIMIT 5;


In [106]:
print(response.metadata["result"])


[('Receta De Ceviche De Chayote', '15 mins', '5 mins', 'Baja'), ('Coliflor Rostizada Con Dip Cremoso', '20 mins', '35 mins', 'Baja'), ('Canoas De Atun', '30 mins', 'N/A', 'Baja'), ('Salsa Martajada Con Tuetano', '50 mins', '15 mins', 'Baja'), ('Ensalada Con Granada Pollo Y Nuez', '20 mins', '15 mins', 'Baja'), ('Pasta Con Chipotle Aguacate Y Mango', '15 mins', '9 mins', 'Baja'), ('Paella De Verduras', '1h', '40 mins', 'Baja'), ('Ensalada Fresca De Melon', '15 mins', 'N/A', 'Baja'), ('Ensalada Mediterranea Con Garbanzos', 'N/A', 'N/A', 'Baja'), ('Lasana De Sushi', '20 mins', '10 mins', 'Baja'), ('Ensalada De Pollo Con Elote Facil', '20 mins', 'N/A', 'Baja'), ('Wrap De Jamon Y Queso', '15 mins', 'N/A', 'Baja'), ('Aguacates Rellenos De Atun', '15 mins', 'N/A', 'Baja'), ('Jugo Verde Facil', 'N/A', 'N/A', 'Baja'), ('Jugo Verde Facil', 'N/A', 'N/A', 'Baja'), ('Galletas Rellenas De Nuez', '40 mins', 'N/A', 'Baja'), ('Ensalada De Camaron', '15 mins', 'N/A', 'Baja'), ('Tinga De Jamaica', '1h', 

In [114]:
print(response.response)


¡Hola! Si estás buscando recetas saludables para ayudarte a bajar de peso, aquí tienes algunas recomendaciones deliciosas:

1. **Receta De Ceviche De Chayote**
   - **Tiempo de preparación:** 15 mins
   - **Tiempo de cocción:** 5 mins
   - **Dificultad:** Baja

2. **Coliflor Rostizada Con Dip Cremoso**
   - **Tiempo de preparación:** 20 mins
   - **Tiempo de cocción:** 35 mins
   - **Dificultad:** Baja

3. **Canoas De Atún**
   - **Tiempo de preparación:** 30 mins
   - **Tiempo de cocción:** N/A
   - **Dificultad:** Baja

4. **Salsa Martajada Con Tuétano**
   - **Tiempo de preparación:** 50 mins
   - **Tiempo de cocción:** 15 mins
   - **Dificultad:** Baja

5. **Ensalada Con Granada, Pollo Y Nuez**
   - **Tiempo de preparación:** 20 mins
   - **Tiempo de cocción:** 15 mins
   - **Dificultad:** Baja

Espero que encuentres alguna de estas recetas útil y deliciosa. ¡Disfruta cocinando! Si necesitas más ayuda, no dudes en preguntar.


In [48]:
import gradio as gr


def model_response(query_str, history=None):
    response = query_engine.query(query_str)
    return response.response


demo = gr.ChatInterface(model_response)

demo.launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


## Vector from Data


In [75]:
import pandas as pd
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import os
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

query = "SELECT * FROM recipes"
# df = pd.read_sql(query, engine)

df.to_csv("data/out/recipes.csv", index=False)
## Create a vector store index FROM the dataframe df

# Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

document = SimpleDirectoryReader(input_files=["data/out/recipes.csv"]).load_data()


In [76]:
vector_store_index = VectorStoreIndex.from_documents(
    documents=document,
)


In [77]:
engine = vector_store_index.as_query_engine()


In [86]:
re = engine.query("Cuantas recetas tienes?")


In [87]:
re.response


'Tienes cuatro recetas en total.'

# Gradio App
